In [1]:
import sys
sys.path.append("models/")

import itertools
import numpy as np
import pandas as pd
import pickle as pkl
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from setup import *
from dataloader import SurveyDataset
import mnl

%load_ext autoreload
%autoreload 2

# Load Model Embeddings

In [2]:
model_type = 'SAE'
load_model_name = 'Autoencoder'
load_model_file = 'sae'
zoomlevel = 'zoom13'
output_dim = 1
model_run_date = '22012701'
model_code = 'M1_D1'

variable_names = ['active','auto','mas','pt', 'trpgen']

demo_variables = ['tot_population','pct25_34yrs','pct35_50yrs','pctover65yrs',
         'pctwhite_alone','pct_nonwhite','pctblack_alone',
         'pct_col_grad','avg_tt_to_work','inc_per_capita']

In [3]:
with open(proj_dir+"latent_space/"+model_type+"_"+zoomlevel+"_"+str(output_dim**2*2048)+"_"+
                       model_run_date+".pkl", "rb") as f:
    encoder_output = pkl.load(f)
    im = pkl.load(f)
    ct = pkl.load(f)

In [4]:
# Aggregate Embeddings
unique_ct = list(set(ct))
unique_ct.sort()
ct = np.array(ct)
aggregate_embeddings = []
for i in unique_ct:
    aggregate_embeddings.append(np.mean(encoder_output[ct == i], axis=0))
aggregate_embeddings = np.array(aggregate_embeddings)

# Load Trip Data

In [5]:
tp = pd.read_csv(data_dir+"trips.csv")
n_alts = 4

In [6]:
print(tp['mode'].value_counts()/len(tp))

2    0.713060
1    0.132001
4    0.111893
3    0.043046
Name: mode, dtype: float64


In [7]:
tp['tract_1'] = tp['state_fips_1'].astype(str) + '_' + tp['county_fips_1'].astype(str)+ '_' + tp['tract_fips_1'].astype(str)
tp['tract_2'] = tp['state_fips_2'].astype(str) + '_' + tp['county_fips_2'].astype(str)+ '_' + tp['tract_fips_2'].astype(str)

tp['morning'] = (tp['dep_hour'] > 6) & (tp['dep_hour'] < 10)
tp['afternoon'] = (tp['dep_hour'] > 15) & (tp['dep_hour'] < 19)
tp['morning'] = tp['morning'].astype(int)
tp['afternoon'] = tp['afternoon'].astype(int)

tp['const'] = 1

def normalize_features(df, cols):
    for c in cols:
        df[c] = df[c]/df[c].max()
    return df

In [8]:
unique_ct = np.array(unique_ct)

x_embed = []
trip_filter = []
for t1, t2 in zip(tp['tract_1'], tp['tract_2']):
    if sum(unique_ct == t1) == 1 and sum(unique_ct == t2) == 1:
        x_embed.append(np.hstack((aggregate_embeddings[unique_ct == t1], aggregate_embeddings[unique_ct == t2])).flatten())
        trip_filter.append(True)
    else:
        trip_filter.append(False)
        

In [9]:
trip_filter = np.array(trip_filter)
x_embed = np.array(x_embed)
x_trip = tp[['morning','afternoon','companion', 'distance', 
         'from_home', 'to_home', 'purp_work', 'purp_school', 'purp_errand', 'purp_recreation', 
         'ontime_important', '12_18yrs', '18_25yrs', '25_55yrs', '55+yrs', 'no_age', 
         'disability', 'educ_col', 'educ_grad', 
         'race_white', 'race_black', 'race_asian', 
         'male', 'female', 
         'emply_park', 'emply_transit', 'emply_veh', 'emply_wfh', 'emply_flex', 'emply_hours', 
         'license', 'person_trips', 'person_transit', 'person_freq_transit', 
         'hh_inc_0_30', 'hh_inc_30_60', 'hh_inc_60_100', 'hh_inc_100_150', 'hh_inc_150', 
         'avg_pr_veh', 'home_own', 'home_house', 'home_condo']].to_numpy()[trip_filter]

x = np.concatenate([x_trip, x_embed], axis=1)

y = tp['mode'].astype(int).to_numpy() - 1
y = y[trip_filter]

In [10]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2)

In [11]:
trainset = SurveyDataset(torch.tensor(x_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.long))
trainloader = DataLoader(trainset, batch_size=256, shuffle=True)

testset = SurveyDataset(torch.tensor(x_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.long))
testloader = DataLoader(testset, batch_size=len(testset), shuffle=False)

In [ ]:
loss_fn = nn.CrossEntropyLoss(reduction='mean')

wd_list = [0.00005,0.0001,0.0005,0.001,0.005,0.01]
lr_list = [0.005, 0.01, 0.02]
do_list = [0, 0.1, 0.2, 0.5]

for (lr, wd, do) in itertools.product(lr_list, wd_list, do_list):

    model = mnl.MNL2(n_alts=n_alts, dim_embed=x_embed.shape[-1], dim_demo=x_trip.shape[-1], dropout=do)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd)

    ref1 = 0
    ref2 = 0

    for epoch in range(100):
        loss_ = 0
        correct = 0
        for batch, (x_batch, y_batch) in enumerate(trainloader):
            # Compute prediction and loss
            util = model(x_batch)
            loss = loss_fn(util, y_batch)
            loss_ += loss.item() * len(x_batch)

            pred = torch.argmax(util, dim=1)
            correct += torch.sum(pred == y_batch)

            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()


        if epoch % 3 == 0:
            loss_ /= len(trainset)
            train_acc = correct/len(trainset)
            print(f"[epoch: {epoch:>3d}] Train loss: {loss_:.4f} accuracy: {train_acc:.3f}")
            
            correct = 0
            test_loss_ = 0
            for batch, (x_batch, y_batch) in enumerate(testloader):
                util = model(x_batch)
                loss = loss_fn(util, y_batch)
                test_loss_ += loss.item()
                pred = torch.argmax(util, dim=1)
                correct += torch.sum(pred == y_batch)
            assert batch == 0 # there is only one batch in test
            test_acc = correct/len(testset)            
            print(f"[epoch: {epoch:>3d}] Test loss: {test_loss_:.4f} accuracy: {test_acc:.3f}")

            if epoch > 15:
                if (np.abs(loss_ - ref1)/ref1<ref1*0.01) & (np.abs(loss_ - ref2)/ref2<ref2*0.01):
                    print("Early stopping at epoch", epoch)
                    break
                if (ref1 < loss_) & (ref1 < ref2):
                    print("Diverging. stop.")
                    break
                if loss_ < best:
                    best = loss_
                    best_test = test_loss_
                    best_epoch = epoch
                    best_train_acc = train_acc
                    best_test_acc = test_acc
            else:
                best = loss_
                best_test = test_loss_
                best_epoch = epoch
                best_train_acc = train_acc
                best_test_acc = test_acc
                
            ref2 = ref1
            ref1 = loss_

    with open(out_dir+model_code+".csv", "a") as f:
        f.write("%s,%s,%s,%s,%.4f,%.5f,%.1f,%d,%.4f,%.4f,%.4f,%.4f\n" % \
            (model_run_date, model_type, zoomlevel, "MNL2", lr, wd, do, 
             best_epoch, best, best_test, best_train_acc, best_test_acc))
        

[epoch:   0] Train loss: 0.5247 accuracy: 0.814
[epoch:   0] Test loss: 0.3965 accuracy: 0.864
[epoch:   3] Train loss: 0.3420 accuracy: 0.881
[epoch:   3] Test loss: 0.3685 accuracy: 0.868
[epoch:   6] Train loss: 0.3216 accuracy: 0.888
[epoch:   6] Test loss: 0.3307 accuracy: 0.884
[epoch:   9] Train loss: 0.3089 accuracy: 0.891
[epoch:   9] Test loss: 0.3018 accuracy: 0.895
[epoch:  12] Train loss: 0.3148 accuracy: 0.889
[epoch:  12] Test loss: 0.3042 accuracy: 0.896
[epoch:  15] Train loss: 0.2993 accuracy: 0.894
[epoch:  15] Test loss: 0.3021 accuracy: 0.897
[epoch:  18] Train loss: 0.2937 accuracy: 0.898
[epoch:  18] Test loss: 0.3009 accuracy: 0.897
[epoch:  21] Train loss: 0.3009 accuracy: 0.894
[epoch:  21] Test loss: 0.3082 accuracy: 0.895
Diverging. stop.
[epoch:   0] Train loss: 0.5089 accuracy: 0.822
[epoch:   0] Test loss: 0.3887 accuracy: 0.866
[epoch:   3] Train loss: 0.3509 accuracy: 0.880
[epoch:   3] Test loss: 0.3375 accuracy: 0.883
[epoch:   6] Train loss: 0.3269 a

[epoch:   9] Train loss: 0.3522 accuracy: 0.875
[epoch:   9] Test loss: 0.3440 accuracy: 0.882
[epoch:  12] Train loss: 0.3404 accuracy: 0.880
[epoch:  12] Test loss: 0.3402 accuracy: 0.884
[epoch:  15] Train loss: 0.3406 accuracy: 0.880
[epoch:  15] Test loss: 0.3400 accuracy: 0.882
[epoch:  18] Train loss: 0.3398 accuracy: 0.880
[epoch:  18] Test loss: 0.3570 accuracy: 0.874
Early stopping at epoch 18
[epoch:   0] Train loss: 0.5385 accuracy: 0.810
[epoch:   0] Test loss: 0.5157 accuracy: 0.811
[epoch:   3] Train loss: 0.3611 accuracy: 0.875
[epoch:   3] Test loss: 0.3503 accuracy: 0.879
[epoch:   6] Train loss: 0.3302 accuracy: 0.885
[epoch:   6] Test loss: 0.3255 accuracy: 0.885
[epoch:   9] Train loss: 0.3239 accuracy: 0.887
[epoch:   9] Test loss: 0.3284 accuracy: 0.883
[epoch:  12] Train loss: 0.3292 accuracy: 0.885
[epoch:  12] Test loss: 0.4481 accuracy: 0.838
[epoch:  15] Train loss: 0.3126 accuracy: 0.892
[epoch:  15] Test loss: 0.3106 accuracy: 0.894
[epoch:  18] Train loss

[epoch:  18] Train loss: 0.4040 accuracy: 0.863
[epoch:  18] Test loss: 0.4009 accuracy: 0.866
[epoch:  21] Train loss: 0.4032 accuracy: 0.864
[epoch:  21] Test loss: 0.3983 accuracy: 0.865
Early stopping at epoch 21
[epoch:   0] Train loss: 0.5440 accuracy: 0.811
[epoch:   0] Test loss: 0.4450 accuracy: 0.846
[epoch:   3] Train loss: 0.4162 accuracy: 0.859
[epoch:   3] Test loss: 0.4025 accuracy: 0.857
[epoch:   6] Train loss: 0.4155 accuracy: 0.859
[epoch:   6] Test loss: 0.4113 accuracy: 0.860
[epoch:   9] Train loss: 0.4108 accuracy: 0.859
[epoch:   9] Test loss: 0.3983 accuracy: 0.867
[epoch:  12] Train loss: 0.4081 accuracy: 0.861
[epoch:  12] Test loss: 0.3956 accuracy: 0.866
[epoch:  15] Train loss: 0.4084 accuracy: 0.861
[epoch:  15] Test loss: 0.4103 accuracy: 0.858
[epoch:  18] Train loss: 0.4085 accuracy: 0.861
[epoch:  18] Test loss: 0.4013 accuracy: 0.860
Early stopping at epoch 18
[epoch:   0] Train loss: 0.5292 accuracy: 0.824
[epoch:   0] Test loss: 0.3843 accuracy: 0.